In [17]:
import numpy as np
import gym
import gym_draughts
from old_game_files.draught_surf import DraughtVisualiser, test

from old_game_files.human_vs_bot import makeboard

ModuleNotFoundError: No module named 'gamefile'

In [12]:
env = gym.make('draughts-v0')

In [13]:
env.render()

[[ 1.  0.  1.  0.  1.  0.]
 [ 0.  1.  0.  1.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [-1.  0. -1.  0. -1.  0.]
 [ 0. -1.  0. -1.  0. -1.]]


In [16]:
test()

SQUARE: 17 22
SQUARE: 13 23
SQUARE: 12 18
SQUARE: 18 17
SQUARE: 15 14
SQUARE: 22 19
SQUARE: 24 15


error: display Surface quit